# **Introduction**

## **Chi-test (boroughs + svi)**
## **Bar-chart with svi as regression/scatterplot (boroughs first)**

source: https://www.atsdr.cdc.gov/place-health/media/pdfs/2024/10/SVI2022Documentation.pdf

source: https://www.atsdr.cdc.gov/place-health/php/svi/svi-interactive-map.html

In [ ]:
# !pip install geopandas folium matplotlib seaborn scipy
# !pip install esda
# !pip install splot
# !pip install geopandas contextily
# # for google colab, had to reinstall some pacakges.

In [ ]:
# !pip install geopandas folium matplotlib seaborn scipy esda splot

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import scipy

# visualization
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns

from scipy.stats import chi2_contingency
import statsmodels.api as sm

# system and utility
import warnings
import os
import io
from IPython.display import IFrame
from google.colab import files

# suppress warnings
warnings.filterwarnings('ignore')

# inline
%matplotlib inline

In [2]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# **Step 1 Get the Eviction data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# data source:
file_path1 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_normal_times_correct_coordinates.csv'
file_path2 = '/content/drive/My Drive/X999/bbl_evictions_311_svi_covid_correct_coordinates.csv'

In [5]:
evictions_pre_post_raw = pd.read_csv(file_path1)
evictions_covid_raw = pd.read_csv(file_path2)
evictions_covid_raw.shape, evictions_pre_post_raw.shape
# correct dimensions

((5366, 102), (66397, 103))

In [6]:
evictions_pre_post = evictions_pre_post_raw.copy()
evictions_covid = evictions_covid_raw.copy()

In [7]:
evictions_pre_post[['latitude', 'longitude']].isna().sum()
# good, the bad bins have been cleaned

,0
latitude,0
longitude,0


In [8]:
bin_100000 = evictions_pre_post[evictions_pre_post['bin'] == 1000000]
bin_200000 = evictions_pre_post[evictions_pre_post['bin'] == 2000000]
bin_300000 = evictions_pre_post[evictions_pre_post['bin'] == 3000000]
bin_400000 = evictions_pre_post[evictions_pre_post['bin'] == 4000000]
bin_500000 = evictions_pre_post[evictions_pre_post['bin'] == 5000000]
bin_600000 = evictions_pre_post[evictions_pre_post['bin'] == 6000000]
bin_100000[['bin', 'primary_key', 'eviction_address', 'latitude', 'longitude']].head()

,bin,primary_key,eviction_address,latitude,longitude
1416,1000000,067758/17_23064,447-448 CENTRAL PARK WEST,40.7973,-73.9606
8259,1000000,300114/23N_116111,517 WEST 134TH STR EET,40.8188,-73.9541
10194,1000000,302057/23_19635,100 WEST 131ST STREET APARTMENT 3C,40.8118,-73.9432
11018,1000000,303174/23_210961,7 DEY STREET A/K/A 185 BROADWAY,40.7106,-74.0100
11081,1000000,303279/22_10840,222 EAST 44TH STREET,40.7518,-73.9728


In [9]:
evictions_pre_post.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count(building),yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,svi_group,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1k(nta),evictions_per_unit_nta_1k(nta),buildings_affected,total_units_per_nta,children_impacted_nta_per_1k,unemployed_impacted_nta_per_1k,elderly_impacted_nta_per_1k,bh_impacted_nta_per_1k,average_year_eviction_per_building_nta(nta)
0,*308072/22_5865,3037420029,*308072/22,5865,356 MILLER AVE,1 AND BASEMENT,2024-12-04,BROOKLYN,11207,Not an Ejectment,Possession,40.6721,-73.8911,5.0000,37.0000,1152.0000,3083989,East New York,2024,2024-12,POINT (-73.891105 40.672121),0.8000,1930.0000,C0,3.0000,3.0000,356 MILLER LLC,2700.0000,pre-war,walk-up,False,low-rise,"Pre-1947, pre-rent-control","1921–1930, Art Deco Skyscrapers","1930-1945, great depression and WWII",3-5 units,True,small,Q3 (50-75%),1930-1939,11207,96801.0000,0.9788,0.9140,0.9808,0.9812,0.9839,33.9000,11.1000,19.1000,6.0000,13.8000,22.5000,13.8000,5.3000,57.8000,9.1000,44.7000,55.9000,32.8000,1.5000,0.0000,0.0000,2.9000,1.6000,94.7000,5.3000,False,Q3,medium-high,0.0000,0.0000,1.0000,0.0000,1.0000,2.0000,0.0000,0.0000,1.0000,0.0000,3.0000,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,0.0000,3.0000,5.0000,19.0000,0.2667,266.0000,2.7479,0.0001,719,50227.0000,0.6183,0.3050,0.3792,2.4374,0.3700
1,*313639/23_5202,3057940012,*313639/23,5202,710 61ST STREET,2ND FLOOR,2024-03-04,BROOKLYN,11220,Not an Ejectment,Possession,40.6359,-74.0119,7.0000,38.0000,118.0000,3143881,Sunset Park East,2024,2024-03,POINT (-74.011883 40.635941),0.6000,1920.0000,B2,2.0000,2.0000,"A.R.M. PARKING, LLC",1204.0000,pre-war,two-family,False,low-rise,"Pre-1947, pre-rent-control","1900–1920, Beaux-Arts","Pre-1929, pre-great depression",2-unit,True,very small,Q1 (smallest 25%),1920-1929,11220,93008.0000,0.9885,0.7635,0.9594,0.9179,0.9662,37.5000,7.5000,37.9000,11.6000,13.1000,25.4000,8.4000,40.2000,61.7000,23.7000,43.6000,1.7000,40.9000,40.7000,0.4000,0.0000,1.2000,0.2000,85.0000,15.0000,False,Q3,medium-high,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,1.0000,1.0000,4.0000,0.3000,38.0000,0.4086,0.0002,136,2144.0000,0.1038,0.0306,0.0535,0.1740,0.2794


In [10]:
evictions_covid.head(2)

,primary_key,bbl,court_index_number,docket_number,eviction_address,eviction_apartment_number,executed_date,borough,zipcode,ejectment,eviction/legal_possession,latitude,longitude,community_board,council_district,census_tract,bin,nta,year,month_year,geometry,average_year_eviction_count(building),yearbuilt,bldgclass,numfloors,unitsres,ownername,bldgarea,building_type,building_category,is_condo,floor_category,rent_era,architectural_style,economic_period,residential_units_category,is_llc,building_size_category,size_quartile,decade,fips,e_totpop,rpl_theme1,rpl_theme2,rpl_theme3,rpl_theme4,rpl_themes,ep_pov150,ep_unemp,ep_nohsdp,ep_uninsur,ep_age65,ep_age17,ep_disabl,ep_limeng,ep_noveh,ep_crowd,ep_hburd,ep_afam,ep_hisp,ep_asian,ep_aian,ep_nhpi,ep_twomore,ep_otherrace,ep_minrty,ep_white,invalid_zip,svi_quartile,air_quality,animal_issues,appliances,building_exterior,doors_windows,electrical_issues,elevator_issues,floors_stairs,general_complaints,graffiti_posting,heat_hot_water,homeless_issues,noise_complaints,other_issues,pest_issues,plumbing_issues,police_matters,public_nuisance,safety_concerns,sanitation_issues,walls_ceilings,total_complaints,average_year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),evictions_per_nta_1k(nta),evictions_per_unit_nta_1k(nta),buildings_affected,total_units_per_nta,children_impacted_nta_per_1k,unemployed_impacted_nta_per_1k,elderly_impacted_nta_per_1k,bh_impacted_nta_per_1k,average_year_eviction_per_building_nta(nta)
0,004123/20_209969,2032140141,004123/20,209969,2541 A GRAND AVE,ROOM 3B,2022-08-22,BRONX,10468,Not an Ejectment,Possession,40.8654,-73.9013,7.0000,14.0000,265.0000,2113173,Kingsbridge Heights,2022,2022-08,POINT (-73.901317 40.865396),0.2000,2004.0000,C0,3.0000,3.0000,MONJU SARKER,3420.0000,post-war,walk-up,False,low-rise,"1994–Present, vacancy decontrol","2001-present, New Architecture","1991–2008, modern economic growth",3-5 units,False,medium-small,Q4 (largest 25%),2000-2009,10468,81397.0000,0.9954,0.9407,0.9870,0.9470,0.9874,39.5000,11.6000,28.3000,9.2000,11.2000,26.4000,12.2000,26.9000,71.8000,19.2000,56.7000,15.6000,78.0000,2.3000,0.0000,0.0000,0.5000,0.5000,96.9000,3.1000,False,Q3,0.0000,0.0000,0.0000,0.0000,3.0000,0.0000,0.0000,2.0000,0.0000,0.0000,1.0000,0.0000,2.0000,0.0000,0.0000,2.0000,0.0000,0.0000,0.0000,3.0000,1.0000,14.0000,0.0667,18.6667,0.1376,0.0001,38,1711.0000,0.0363,0.0160,0.0154,0.1288,0.4912
1,0050153/20_106030,4031560133,0050153/20,106030,98-05 67TH AVENUE,12F,2022-04-14,QUEENS,11375,Not an Ejectment,Possession,40.7242,-73.8556,6.0000,29.0000,71306.0000,4074666,Forest Hills,2022,2022-04,POINT (-73.855552 40.724241),0.2000,1960.0000,D3,13.0000,181.0000,MARSEILLES LEASING LIMITED PARTNERSHIP,177710.0000,post-war,elevator,False,high-rise,"1947–1969, rent-control","1951–1980, the International Style, Alternativ...","1946–1975, pst war economic boom",100+ units,False,mega,Q4 (largest 25%),1960-1969,11375,75212.0000,0.4759,0.5698,0.8789,0.8057,0.7322,12.0000,4.8000,6.1000,3.7000,20.4000,18.0000,10.5000,7.9000,41.9000,5.8000,25.4000,2.7000,16.4000,28.5000,0.1000,0.0000,4.6000,0.7000,53.0000,47.0000,False,Q1 (Low),0.0000,2.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.0000,0.0000,62.0000,0.0000,34.0000,0.0000,0.0000,4.0000,1.0000,0.0000,0.0000,2.0000,5.0000,112.0000,0.0011,6.0000,0.0479,0.0000,17,2403.0000,0.0086,0.0023,0.0098,0.0091,0.3529


In [11]:
evictions_pre_post.columns, \
evictions_covid.columns, \
evictions_pre_post.shape, \
evictions_covid.shape

(Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment',
        ...
        'average_year_eviction_count_per_nta(nta)', 'evictions_per_nta_1k(nta)',
        'evictions_per_unit_nta_1k(nta)', 'buildings_affected',
        'total_units_per_nta', 'children_impacted_nta_per_1k',
        'unemployed_impacted_nta_per_1k', 'elderly_impacted_nta_per_1k',
        'bh_impacted_nta_per_1k',
        'average_year_eviction_per_building_nta(nta)'],
       dtype='object', length=103),
 Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
        'eviction_address', 'eviction_apartment_number', 'executed_date',
        'borough', 'zipcode', 'ejectment',
        ...
        'average_year_eviction_count_per_nta(nta)', 'evictions_per_nta_1k(nta)',
        'evictions_per_unit_nta_1k(nta)', 'buildings_affected',
        'total_units_per_nta', 'children_impacted_nt

In [12]:
link = '/content/drive/My Drive/X999/svi_cleaned.csv'

In [13]:
svi_df = pd.read_csv(link)
svi_df.head(2)

,fips,location,area_sqmi,e_totpop,m_totpop,e_hu,m_hu,e_hh,m_hh,e_pov150,m_pov150,e_unemp,m_unemp,e_hburd,m_hburd,e_nohsdp,m_nohsdp,e_uninsur,m_uninsur,e_age65,m_age65,e_age17,m_age17,e_disabl,m_disabl,e_sngpnt,m_sngpnt,e_limeng,m_limeng,e_minrty,m_minrty,e_munit,m_munit,e_mobile,m_mobile,e_crowd,m_crowd,e_noveh,m_noveh,e_groupq,m_groupq,ep_pov150,mp_pov150,ep_unemp,mp_unemp,ep_hburd,mp_hburd,ep_nohsdp,mp_nohsdp,ep_uninsur,mp_uninsur,ep_age65,mp_age65,ep_age17,mp_age17,ep_disabl,mp_disabl,ep_sngpnt,mp_sngpnt,ep_limeng,mp_limeng,ep_minrty,mp_minrty,ep_munit,mp_munit,ep_mobile,mp_mobile,ep_crowd,mp_crowd,ep_noveh,mp_noveh,ep_groupq,mp_groupq,epl_pov150,epl_unemp,epl_hburd,epl_nohsdp,epl_uninsur,spl_theme1,rpl_theme1,epl_age65,epl_age17,epl_disabl,epl_sngpnt,epl_limeng,spl_theme2,rpl_theme2,epl_minrty,spl_theme3,rpl_theme3,epl_munit,epl_mobile,epl_crowd,epl_noveh,epl_groupq,spl_theme4,rpl_theme4,spl_themes,rpl_themes,f_pov150,f_unemp,f_hburd,f_nohsdp,f_uninsur,f_theme1,f_age65,f_age17,f_disabl,f_sngpnt,f_limeng,f_theme2,f_minrty,f_theme3,f_munit,f_mobile,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_daypop,e_noint,m_noint,e_afam,m_afam,e_hisp,m_hisp,e_asian,m_asian,e_aian,m_aian,e_nhpi,m_nhpi,e_twomore,m_twomore,e_otherrace,m_otherrace,ep_noint,mp_noint,ep_afam,mp_afam,ep_hisp,mp_hisp,ep_asian,mp_asian,ep_aian,mp_aian,ep_nhpi,mp_nhpi,ep_twomore,mp_twomore,ep_otherrace,mp_otherrace
0,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000
1,10002,ZCTA5 10002,0.8223,76518,2894,39094,1241,36028,1326,27908,2853,2833,574,14688,1367,18301,1376,4074,766,17681,1287,10028,1549,9896,1062,2211,499,18393,1640,56964,3226,35725,1677,16,28,2461,449,29828,1403,2090,39,36.8000,3.5000,7.6000,1.4000,40.8000,3.5000,30.0000,2.0000,5.4000,1.0000,23.1000,1.7000,13.1000,1.8000,13.0000,1.4000,6.1000,1.4000,24.7000,2.0000,74.4000,3.1000,91.4000,3.2000,0.0000,0.1000,6.8000,1.2000,82.8000,1.8000,2.7000,0.1000,0.9148,0.7946,0.9219,0.9741,0.7207,4.3261,0.9639,0.7296,0.1831,0.5186,0.7390,0.9944,3.1647,0.8781,0.9369,0.9369,0.9369,0.9790,0.0000,0.9105,0.9915,0.7730,3.6540,0.9254,12.0817,0.9656,1,0,1,1,0,3,0,0,0,0,1,1,1,1,1,0,1,1,0,3,8,64307,8590,1110,6141,1194,19864,2190,28477,1989,74,83,24,45,1810,486,574,394,23.8000,2.9000,8.0000,1.5000,26.0000,2.5000,37.2000,2.2000,0.1000,0.1000,0.0000,0.1000,2.4000,0.6000,0.8000,0.5000


In [14]:
svi_df.shape

(204, 153)

In [15]:
# list(svi_df.columns)

In [16]:
svi_df.ep_nhpi.unique()

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

# **Step 2: SVI items**

q quick double check

In [17]:
link = "/content/drive/My Drive/X999/NewYork_ZCTA.csv"

In [18]:
svi_raw = pd.read_csv(link)
svi_raw.head(2)

,ST,STATE,ST_ABBR,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,36,New York,NY,6390,ZCTA5 06390,4.0467,53,39,253,49,19,19,17,16,0,13,9,26,0,13,27,34,0,13,6,11,31,33,0,18,9,53,20,51,0,18,4,5,0,18,0,13,17,16,32.1000,18.8000,0.0000,52.7000,47.4000,100.0000,0.0000,51.4000,50.9000,45.7000,0.0000,45.2000,11.3000,19.0000,58.5000,41.4000,0.0000,94.7000,17.0000,99.2000,37.7000,92.1000,0.0000,7.1000,1.6000,2.0000,0.0000,94.7000,0.0000,75.5000,32.1000,18.8000,0.8790,0.0000,0.9635,0.0000,0.9960,2.8385,0.6342,0.0000,0.1408,0.9944,0.0000,0.9775,2.1127,0.3009,0.8062,0.8062,0.8062,0.0000,0.4654,0.0000,0.0000,0.9735,1.4389,0.2205,7.1963,0.4192,0,0,1,0,1,2,0,0,1,0,1,2,0,0,0,0,0,0,1,1,5,601,9,14,0,13,9,19,0,13,0,13,8,16,3,7,0,13,47.4000,51.8000,0.0000,45.2000,17.0000,35.0000,0.0000,45.2000,0.0000,45.2000,15.1000,32.1000,5.7000,12.1000,0.0000,45.2000
1,36,New York,NY,10001,ZCTA5 10001,0.6238,27004,1827,16975,831,14375,782,5248,797,761,266,3314,531,1930,534,831,289,3428,432,2694,643,2310,499,501,215,1381,405,13460,2305,15840,898,15,23,389,135,12285,840,2213,218,20.3000,2.7000,4.3000,1.5000,23.1000,3.5000,9.1000,2.4000,3.1000,1.0000,12.7000,1.6000,10.0000,2.1000,8.6000,1.9000,3.5000,1.5000,5.3000,1.5000,49.8000,7.8000,93.3000,2.7000,0.1000,0.1000,2.7000,0.9000,85.5000,2.8000,8.2000,0.6000,0.6108,0.4574,0.5573,0.5902,0.4436,2.6593,0.5688,0.1420,0.1161,0.1891,0.4707,0.8777,1.7956,0.1692,0.8670,0.8670,0.8670,0.9853,0.2710,0.7402,0.9949,0.9104,3.9018,0.9806,9.2237,0.7414,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,3,3,239407,1047,389,2220,576,5206,943,5031,774,0,25,0,25,780,326,223,169,7.3000,2.6000,8.2000,2.2000,19.3000,3.0000,18.6000,2.9000,0.0000,0.1000,0.0000,0.1000,2.9000,1.2000,0.8000,0.6000


In [19]:
def is_nyc_zipcode(zipcode):
    zip_int = int(zipcode) if isinstance(zipcode, str) else zipcode

    # Manhattan: 10001-10282
    if 10001 <= zip_int <= 10282:
        return True
    # addition Manhattan: 10300-10499
    if 10300 <= zip_int <= 10499:
        return True
    # Bronx: 10451-10475
    if 10451 <= zip_int <= 10475:
        return True
    # Brooklyn: 11201-11256
    if 11201 <= zip_int <= 11256:
        return True
    # Queens: 11351-11436, 11101-11109
    if (11351 <= zip_int <= 11436) or (11101 <= zip_int <= 11109):
        return True
    # Staten Island: 10301-10314
    if 10301 <= zip_int <= 10314:
        return True
    # additional Queens ZIPs
    if zip_int in [11004, 11005, 11411, 11412, 11413, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429]:
        return True
    return False

In [20]:
nyc_df = svi_raw[svi_raw['FIPS'].apply(is_nyc_zipcode)]

In [21]:
nyc_df.shape

(204, 156)

In [22]:
nyc_df.EP_NHPI.unique()
# interesting, so there was an error in the source data

array([ 0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,  8.00e-01,
        1.20e+00,  5.00e-01,  4.00e-01])

In [23]:
svi_raw.EP_NHPI.unique()

array([ 1.51e+01,  0.00e+00,  1.00e-01,  3.00e-01,  2.00e-01, -9.99e+02,
        8.00e-01,  1.20e+00,  5.00e-01,  4.00e-01,  1.10e+00,  7.00e-01,
        1.50e+00,  1.80e+00,  9.00e-01,  6.00e-01,  1.40e+00,  2.10e+00,
        2.30e+00,  1.00e+00])

In [24]:
-9.99e+02, 0.00e+00, 9.00e-01, 2.00e-01

(-999.0, 0.0, 0.9, 0.2)

# **Step 3:  Boroughs and their eviction rates**

In [25]:
evictions_pre_post_mean = evictions_pre_post[['ep_afam', 'ep_asian', 'ep_hisp', 'ep_nhpi', 'ep_white', 'ep_twomore', 'ep_otherrace']].mean()
evictions_pre_post_mean
# may need to merge ep_twomore and ep_otherrance toegther

,0
ep_afam,29.2346
ep_asian,8.9040
ep_hisp,38.7307
ep_nhpi,0.0115
ep_white,19.0236
ep_twomore,2.8025
ep_otherrace,1.0239


In [26]:
evictions_pre_post_mean = evictions_pre_post_mean.reset_index()

In [27]:
evictions_pre_post_mean.rename(columns = {'index':'race_svi', 0: "racial percentage"}, inplace=True)

In [28]:
evictions_pre_post_mean

,race_svi,racial percentage
0,ep_afam,29.2346
1,ep_asian,8.9040
2,ep_hisp,38.7307
3,ep_nhpi,0.0115
4,ep_white,19.0236
5,ep_twomore,2.8025
6,ep_otherrace,1.0239


In [29]:
# type(evictions_pre_post_mean)
# so this is correct

## **Step 3.1 Selected neighborhoods**

four types of aggregated and groupedby analysis:

1) average eviction count per building, per year;

2) average eviction count per nta population;

3) average eviction count per unit per building per year;

4) average eviction count per unit per nta population.

In [ ]:
normal[['average_year_eviction_count(building)', 'average_year_eviction_count_per_unit(building)', 'average_year_eviction_count_per_nta(nta)',
        'average_year_eviction_per_building_nta(nta)', 'evictions_per_nta_1k(nta)', 'evictions_per_unit_nta_1k(nta)']]

In [33]:
analysis_columns = evictions_pre_post[['primary_key', 'bin', 'bbl', 'latitude', 'longitude', 'eviction_address', 'zipcode', 'borough', 'nta','average_year_eviction_count(building)', 'average_year_eviction_count_per_unit(building)', 'average_year_eviction_count_per_nta(nta)',
        'average_year_eviction_per_building_nta(nta)', 'evictions_per_nta_1k(nta)', 'evictions_per_unit_nta_1k(nta)',
                              'unitsres', 'e_totpop']]
analysis_columns.head()
# average_year_eviction_nta_count is (total evictions per nta) / (nta's population)

,primary_key,bin,bbl,latitude,longitude,eviction_address,zipcode,borough,nta,average_year_eviction_count(building),average_year_eviction_count_per_unit(building),average_year_eviction_count_per_nta(nta),average_year_eviction_per_building_nta(nta),evictions_per_nta_1k(nta),evictions_per_unit_nta_1k(nta),unitsres,e_totpop
0,*308072/22_5865,3083989,3037420029,40.6721,-73.8911,356 MILLER AVE,11207,BROOKLYN,East New York,0.8000,0.2667,266.0000,0.3700,2.7479,0.0001,3.0000,96801.0000
1,*313639/23_5202,3143881,3057940012,40.6359,-74.0119,710 61ST STREET,11220,BROOKLYN,Sunset Park East,0.6000,0.3000,38.0000,0.2794,0.4086,0.0002,2.0000,93008.0000
2,*324973/22_5308,3143435,3057820030,40.6400,-74.0171,462 60TH STREET,11220,BROOKLYN,Sunset Park West,0.6000,0.1500,39.6000,0.2694,0.4258,0.0003,4.0000,93008.0000
3,*53336/16_170279,2015444,2032510420,40.8772,-73.8896,3400 PAUL AVENUE,10468,BRONX,Van Cortlandt Village,0.8000,0.0023,172.8000,0.7082,2.1229,0.0001,352.0000,81397.0000
4,*5990/17_2703,2003900,2025770038,40.8112,-73.9088,480 CONCORD AVENUE,10455,BRONX,Mott Haven-Port Morris,1.6000,0.0246,158.8000,0.4932,3.5782,0.0001,65.0000,44380.0000


In [34]:
evictions_pre_post.columns

Index(['primary_key', 'bbl', 'court_index_number', 'docket_number',
       'eviction_address', 'eviction_apartment_number', 'executed_date',
       'borough', 'zipcode', 'ejectment',
       ...
       'average_year_eviction_count_per_nta(nta)', 'evictions_per_nta_1k(nta)',
       'evictions_per_unit_nta_1k(nta)', 'buildings_affected',
       'total_units_per_nta', 'children_impacted_nta_per_1k',
       'unemployed_impacted_nta_per_1k', 'elderly_impacted_nta_per_1k',
       'bh_impacted_nta_per_1k',
       'average_year_eviction_per_building_nta(nta)'],
      dtype='object', length=103)

In [37]:
neighbor_evictions = evictions_pre_post.groupby('nta').agg({'average_year_eviction_count_per_nta(nta)': 'mean',
                                                            'ep_afam': 'first',
                                                            'ep_asian': 'first',
                                                            'ep_hisp': 'first',
                                                            'ep_nhpi': 'first',
                                                            'ep_white': 'first',
                                                            'ep_twomore': 'first',
                                                            'ep_otherrace': 'first',
                                                            'rpl_themes': 'first',
                                                            'borough': 'first',
                                                            }).reset_index()
neighbor_evictions.sort_values('average_year_eviction_count_per_nta(nta)', ascending=False, inplace=True)
neighbor_evictions

,nta,average_year_eviction_count_per_nta(nta),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes,borough
27,Central Harlem North-Polo Grounds,326.4000,54.7000,1.9000,32.7000,0.0000,6.7000,2.8000,1.1000,0.9748,MANHATTAN
38,Crown Heights North,319.8000,66.7000,1.3000,15.7000,0.0000,11.9000,3.6000,0.2000,0.9438,BROOKLYN
11,Bedford Park-Fordham North,306.0000,15.6000,2.3000,78.0000,0.0000,3.1000,0.5000,0.5000,0.9874,BRONX
44,East Concourse-Concourse Village,287.6000,38.2000,0.7000,56.3000,0.0000,2.4000,1.8000,0.4000,0.9994,BRONX
178,Williamsbridge-Olinville,267.4000,52.2000,5.5000,29.5000,0.0000,8.8000,2.3000,1.0000,0.9507,BRONX
50,East New York,266.0000,55.9000,1.5000,32.8000,0.0000,5.3000,2.9000,1.6000,0.9839,BROOKLYN
163,University Heights-Morris Heights,265.6000,24.6000,1.6000,69.2000,0.0000,2.0000,1.9000,0.7000,0.9983,BRONX
108,Mount Hope,260.2000,30.9000,0.4000,65.3000,0.0000,1.7000,1.1000,0.4000,0.9989,BRONX
59,Flatbush,246.2000,51.6000,5.8000,7.5000,0.0000,29.7000,4.3000,1.2000,0.9174,BROOKLYN
52,East Tremont,219.8000,26.5000,1.4000,65.0000,0.0000,3.8000,1.9000,1.0000,0.9954,BRONX


In [38]:
top_15 = neighbor_evictions.head(15)
bottom_15 = neighbor_evictions.tail(15)
selected_nta = pd.concat([top_15, bottom_15])
selected_nta

,nta,average_year_eviction_count_per_nta(nta),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes,borough
27,Central Harlem North-Polo Grounds,326.4000,54.7000,1.9000,32.7000,0.0000,6.7000,2.8000,1.1000,0.9748,MANHATTAN
38,Crown Heights North,319.8000,66.7000,1.3000,15.7000,0.0000,11.9000,3.6000,0.2000,0.9438,BROOKLYN
11,Bedford Park-Fordham North,306.0000,15.6000,2.3000,78.0000,0.0000,3.1000,0.5000,0.5000,0.9874,BRONX
44,East Concourse-Concourse Village,287.6000,38.2000,0.7000,56.3000,0.0000,2.4000,1.8000,0.4000,0.9994,BRONX
178,Williamsbridge-Olinville,267.4000,52.2000,5.5000,29.5000,0.0000,8.8000,2.3000,1.0000,0.9507,BRONX
50,East New York,266.0000,55.9000,1.5000,32.8000,0.0000,5.3000,2.9000,1.6000,0.9839,BROOKLYN
163,University Heights-Morris Heights,265.6000,24.6000,1.6000,69.2000,0.0000,2.0000,1.9000,0.7000,0.9983,BRONX
108,Mount Hope,260.2000,30.9000,0.4000,65.3000,0.0000,1.7000,1.1000,0.4000,0.9989,BRONX
59,Flatbush,246.2000,51.6000,5.8000,7.5000,0.0000,29.7000,4.3000,1.2000,0.9174,BROOKLYN
52,East Tremont,219.8000,26.5000,1.4000,65.0000,0.0000,3.8000,1.9000,1.0000,0.9954,BRONX


In [39]:
# get rid of the two cemetery:
selected_nta = selected_nta[~selected_nta['nta'].isin(['park-cemetery-etc-Brooklyn', 'park-cemetery-etc-Bronx'])]
selected_nta.shape

(28, 11)

In [40]:
selected_nta['black+hispanic'] = selected_nta['ep_hisp'] + selected_nta['ep_nhpi']

In [41]:
selected_nta.head()

,nta,average_year_eviction_count_per_nta(nta),ep_afam,ep_asian,ep_hisp,ep_nhpi,ep_white,ep_twomore,ep_otherrace,rpl_themes,borough,black+hispanic
27,Central Harlem North-Polo Grounds,326.4000,54.7000,1.9000,32.7000,0.0000,6.7000,2.8000,1.1000,0.9748,MANHATTAN,32.7000
38,Crown Heights North,319.8000,66.7000,1.3000,15.7000,0.0000,11.9000,3.6000,0.2000,0.9438,BROOKLYN,15.7000
11,Bedford Park-Fordham North,306.0000,15.6000,2.3000,78.0000,0.0000,3.1000,0.5000,0.5000,0.9874,BRONX,78.0000
44,East Concourse-Concourse Village,287.6000,38.2000,0.7000,56.3000,0.0000,2.4000,1.8000,0.4000,0.9994,BRONX,56.3000
178,Williamsbridge-Olinville,267.4000,52.2000,5.5000,29.5000,0.0000,8.8000,2.3000,1.0000,0.9507,BRONX,29.5000


In [45]:
selected_nta.T

,27,38,11,44,178,50,163,108,59,52,169,132,106,21,171,118,177,4,176,29,180,134,42,20,2,1,68,140
nta,Central Harlem North-Polo Grounds,Crown Heights North,Bedford Park-Fordham North,East Concourse-Concourse Village,Williamsbridge-Olinville,East New York,University Heights-Morris Heights,Mount Hope,Flatbush,East Tremont,Washington Heights South,Prospect Lefferts Gardens-Wingate,Morrisania-Melrose,Brownsville,West Concourse,Oakland Gardens,Whitestone,Auburndale,Westerleigh,Charleston-Richmond Valley-Tottenville,Windsor Terrace,Queensboro Hill,Douglas Manor-Douglaston-Little Neck,Brooklyn Heights-Cobble Hill,Arden Heights,Annadale-Huguenot-Prince's Bay-Eltingville,Glen Oaks-Floral Park-New Hyde Park,Rossville-Woodrow
average_year_eviction_count_per_nta(nta),326.4000,319.8000,306.0000,287.6000,267.4000,266.0000,265.6000,260.2000,246.2000,219.8000,209.2000,204.2000,202.0000,193.6000,182.4000,7.6000,7.2000,7.2000,6.8000,6.2000,6.2000,6.2000,5.0000,4.8000,3.4000,3.4000,3.4000,3.4000
ep_afam,54.7000,66.7000,15.6000,38.2000,52.2000,55.9000,24.6000,30.9000,51.6000,26.5000,12.8000,63.2000,38.2000,70.1000,30.9000,1.6000,0.3000,11.2000,3.3000,0.1000,8.9000,11.2000,0.3000,11.4000,0.8000,0.5000,5.7000,0.5000
ep_asian,1.9000,1.3000,2.3000,0.7000,5.5000,1.5000,1.6000,0.4000,5.8000,1.4000,4.5000,3.2000,0.7000,0.8000,0.4000,48.6000,26.0000,43.0000,18.0000,2.1000,17.8000,43.0000,30.9000,14.2000,8.7000,4.1000,46.8000,4.1000
ep_hisp,32.7000,15.7000,78.0000,56.3000,29.5000,32.8000,69.2000,65.3000,7.5000,65.0000,65.7000,14.9000,56.3000,19.5000,65.3000,16.4000,14.8000,21.2000,15.2000,10.6000,16.4000,21.2000,12.4000,11.0000,12.8000,9.6000,18.0000,9.6000
ep_nhpi,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2000,0.0000,0.0000,0.1000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
ep_white,6.7000,11.9000,3.1000,2.4000,8.8000,5.3000,2.0000,1.7000,29.7000,3.8000,14.6000,13.7000,2.4000,3.5000,1.7000,30.2000,56.1000,22.2000,60.3000,86.0000,51.5000,22.2000,51.3000,57.5000,76.1000,83.5000,23.0000,83.5000
ep_twomore,2.8000,3.6000,0.5000,1.8000,2.3000,2.9000,1.9000,1.1000,4.3000,1.9000,2.0000,4.1000,1.8000,5.4000,1.1000,2.5000,1.9000,1.9000,2.4000,1.2000,4.6000,1.9000,3.8000,5.3000,1.2000,1.5000,4.1000,1.5000
ep_otherrace,1.1000,0.2000,0.5000,0.4000,1.0000,1.6000,0.7000,0.4000,1.2000,1.0000,0.4000,0.7000,0.4000,0.6000,0.4000,0.3000,0.5000,0.4000,0.5000,0.0000,0.9000,0.4000,0.3000,0.5000,0.4000,0.7000,2.4000,0.7000
rpl_themes,0.9748,0.9438,0.9874,0.9994,0.9507,0.9839,0.9983,0.9989,0.9174,0.9954,0.9771,0.9220,0.9994,0.9948,0.9989,0.8240,0.8561,0.9077,0.7861,0.3624,0.9197,0.9077,0.7987,0.6233,0.5333,0.5642,0.7093,0.5642


In [43]:
selected_nta_excel = selected_nta[['nta', 'average_year_eviction_count_per_nta(nta)', 'black+hispanic', 'rpl_themes', 'borough']]
selected_nta_excel

,nta,average_year_eviction_count_per_nta(nta),black+hispanic,rpl_themes,borough
27,Central Harlem North-Polo Grounds,326.4000,32.7000,0.9748,MANHATTAN
38,Crown Heights North,319.8000,15.7000,0.9438,BROOKLYN
11,Bedford Park-Fordham North,306.0000,78.0000,0.9874,BRONX
44,East Concourse-Concourse Village,287.6000,56.3000,0.9994,BRONX
178,Williamsbridge-Olinville,267.4000,29.5000,0.9507,BRONX
50,East New York,266.0000,32.8000,0.9839,BROOKLYN
163,University Heights-Morris Heights,265.6000,69.2000,0.9983,BRONX
108,Mount Hope,260.2000,65.3000,0.9989,BRONX
59,Flatbush,246.2000,7.5000,0.9174,BROOKLYN
52,East Tremont,219.8000,65.0000,0.9954,BRONX


groups, group that has evictions over 100-200, and then we have the group of nta and then we would have the 
mean of svi of those nta (and therefore the quartile). for each grouped nta, we get the svi as bar and whisker. 
outlier of each group, and see the trends of bars going up when the avergae evictions go over (as x-axis grows) 
EXCEL. 

do the columns and then just use one type(bar and whiskers). 

In [44]:
selected_nta_excel.T

,27,38,11,44,178,50,163,108,59,52,169,132,106,21,171,118,177,4,176,29,180,134,42,20,2,1,68,140
nta,Central Harlem North-Polo Grounds,Crown Heights North,Bedford Park-Fordham North,East Concourse-Concourse Village,Williamsbridge-Olinville,East New York,University Heights-Morris Heights,Mount Hope,Flatbush,East Tremont,Washington Heights South,Prospect Lefferts Gardens-Wingate,Morrisania-Melrose,Brownsville,West Concourse,Oakland Gardens,Whitestone,Auburndale,Westerleigh,Charleston-Richmond Valley-Tottenville,Windsor Terrace,Queensboro Hill,Douglas Manor-Douglaston-Little Neck,Brooklyn Heights-Cobble Hill,Arden Heights,Annadale-Huguenot-Prince's Bay-Eltingville,Glen Oaks-Floral Park-New Hyde Park,Rossville-Woodrow
average_year_eviction_count_per_nta(nta),326.4000,319.8000,306.0000,287.6000,267.4000,266.0000,265.6000,260.2000,246.2000,219.8000,209.2000,204.2000,202.0000,193.6000,182.4000,7.6000,7.2000,7.2000,6.8000,6.2000,6.2000,6.2000,5.0000,4.8000,3.4000,3.4000,3.4000,3.4000
black+hispanic,32.7000,15.7000,78.0000,56.3000,29.5000,32.8000,69.2000,65.3000,7.5000,65.0000,65.7000,14.9000,56.3000,19.5000,65.3000,16.4000,15.0000,21.2000,15.2000,10.7000,16.4000,21.2000,12.4000,11.0000,12.8000,9.6000,18.0000,9.6000
rpl_themes,0.9748,0.9438,0.9874,0.9994,0.9507,0.9839,0.9983,0.9989,0.9174,0.9954,0.9771,0.9220,0.9994,0.9948,0.9989,0.8240,0.8561,0.9077,0.7861,0.3624,0.9197,0.9077,0.7987,0.6233,0.5333,0.5642,0.7093,0.5642
borough,MANHATTAN,BROOKLYN,BRONX,BRONX,BRONX,BROOKLYN,BRONX,BRONX,BROOKLYN,BRONX,MANHATTAN,BROOKLYN,BRONX,BROOKLYN,BRONX,QUEENS,QUEENS,QUEENS,STATEN ISLAND,STATEN ISLAND,BROOKLYN,QUEENS,QUEENS,BROOKLYN,STATEN ISLAND,STATEN ISLAND,QUEENS,STATEN ISLAND
